# Importing Libraries

In [52]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.subplots as subplots
import cufflinks as cf
%matplotlib inline
from plotly.offline import iplot
import plotly.graph_objs as go

In [53]:
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

# Getting Data

In [54]:
!gdown --id 1x3gfhI0kX-tckRoQRENF_jTCFLgILCuM

Downloading...
From: https://drive.google.com/uc?id=1x3gfhI0kX-tckRoQRENF_jTCFLgILCuM
To: /content/archive.zip
100% 1.43M/1.43M [00:00<00:00, 94.9MB/s]


In [55]:
!unzip '/content/archive.zip' -d '/content/data'

Archive:  /content/archive.zip
replace /content/data/IPL Ball-by-Ball 2008-2020.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [56]:
df_balls = pd.read_csv("/content/data/IPL Ball-by-Ball 2008-2020.csv")
print(df_balls.shape)
#df_balls.head()


(193468, 18)


In [57]:
df_matches = pd.read_csv("/content/data/IPL Matches 2008-2020.csv")
print(df_matches.shape)
#df_matches.head()

(816, 17)


In [58]:
data = pd.merge(left=df_matches, right=df_balls, on='id', how='right')
print(data.shape)
data.head()

(193468, 34)


,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


# Formatting

**Adding another column for year**

In [59]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = pd.DatetimeIndex(data['date']).year
data.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,year
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
2,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
3,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
4,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008


**Making overs run from 1 to 20 instead of 0 to 19**

In [60]:
print(np.sort(data['over'].unique()))
data['over'] = data['over']+1
print(np.sort(data['over'].unique()))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


**Correcting a duplicate entry in teams' names**

In [61]:
data['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals'], dtype=object)

In [62]:
data.replace({'Rising Pune Supergiants' : 'Rising Pune Supergiant'},regex=True,inplace=True)
df_balls.replace({'Rising Pune Supergiants' : 'Rising Pune Supergiant'},regex=True,inplace=True)
df_matches.replace({'Rising Pune Supergiants' : 'Rising Pune Supergiant'},regex=True,inplace=True)

**Correcting spelling of Bengalore to Bengaluru for one entry of city**

In [63]:
data.replace({'Bangalore':'Bengaluru'},regex=True,inplace=True)
df_balls.replace({'Bangalore':'Bengaluru'},regex=True,inplace=True)
df_matches.replace({'Bangalore':'Bengaluru'},regex=True,inplace=True)

In [64]:
data['team1'].unique()

array(['Royal Challengers Bengaluru', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiant', 'Delhi Capitals'],
      dtype=object)

# Runs scored over the years

In [65]:
runs_year = data.groupby(by='year').sum()['total_runs']
runs_year = pd.DataFrame(runs_year)
runs_year = runs_year.reset_index(drop=False)
runs_year.head()

,year,total_runs
0,2008,17937
1,2009,16320
2,2010,18864
3,2011,21154
4,2012,22453


In [66]:
import random
def random_colors(number_of_colors):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
    return color

In [67]:
total_runs = go.Bar(
                    x=runs_year['year'],
                    y=runs_year['total_runs'],
                    #mode='lines',
                    marker = dict(color = random_colors(len(runs_year['year'].value_counts().index)),line=dict(color='#000000', width=1)),

                    name='runs'
                    )
layout = go.Layout(autosize=False,
                      width=800,
                      height=400,
                  title='Runs scored per year',
                  xaxis = dict(title='Year'),
                  yaxis = dict(title='Runs'))

fig = go.Figure(data=[total_runs], layout=layout)

pyo.iplot(fig)

# Toss Decisions

In [68]:
df_matches.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bengaluru,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bengaluru,Kolkata Knight Riders,Royal Challengers Bengaluru,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bengaluru,Mumbai Indians,bat,Royal Challengers Bengaluru,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [69]:
toss_data = pd.DataFrame(df_matches['toss_decision'].value_counts()).reset_index()
display(toss_data.head())
toss_data2 = pd.DataFrame()
toss_data2['toss_influence'] = df_matches.apply(lambda x : 'Yes' if x['toss_winner']==x['winner'] else 'No', axis=1)
toss_data2 = toss_data2.value_counts().reset_index()
toss_data2.head()

,index,toss_decision
0,field,496
1,bat,320


,toss_influence,0
0,Yes,418
1,No,398


In [70]:
toss_plot = go.Bar(
                    y=toss_data['index'],
                    x=toss_data['toss_decision'],
                   orientation='h',
                   marker_color=px.colors.qualitative.Dark24[0:2],
                   #color_discrete_sequence=["red"]
                    #name='runs'
                    )
layout = go.Layout(   autosize=False,
                      width=300,
                      height=300,
                   #colorscale = px.colors.sequential.Viridis,
                      #color='toss_decision',
                      title='Toss Decisions',
                      yaxis = dict(title='Decision'),
                      xaxis = dict(title='Counts'),
                      margin=go.layout.Margin(
                                                    l=50,
                                                    r=50,
                                                    b=100,
                                                    t=100,
                                                    pad = 4
                                                )
                       )
fig = go.Figure(data=[toss_plot], layout=layout)

pyo.iplot(fig)

In [72]:
toss_plot2 = go.Bar(
                    y=toss_data2['toss_influence'],
                    x=toss_data2[0],
                   orientation='h',
                   marker_color=px.colors.qualitative.Plotly[0:2],
                   #color_discrete_sequence=["red"]
                    #name='runs'
                    )
layout = go.Layout(   autosize=False,
                      width=350,
                      height=350,
                   #colorscale = px.colors.sequential.Viridis,
                      #color='toss_decision',
                      title='Does Wining toss helps teams to win?',
                      xaxis = dict(title='Counts'),
                      yaxis = dict(title='Is influencial'),
                      margin=go.layout.Margin(
                                                    l=50,
                                                    r=50,
                                                    b=100,
                                                    t=100,
                                                    pad = 4
                                                )
                       )
fig = go.Figure(data=[toss_plot2], layout=layout)

pyo.iplot(fig)

# Runs and Wickets per over

In [ ]:
runs_and_wickets = data.groupby(by='over').sum()[['total_runs','is_wicket']].reset_index()
runs_and_wickets['total_runs'] = (runs_and_wickets['total_runs']-runs_and_wickets['total_runs'].min())/(runs_and_wickets['total_runs'].max()-runs_and_wickets['total_runs'].min())
runs_and_wickets['is_wicket'] = (runs_and_wickets['is_wicket']-runs_and_wickets['is_wicket'].min())/(runs_and_wickets['is_wicket'].max()-runs_and_wickets['is_wicket'].min())
runs_and_wickets.head()

,over,total_runs,is_wicket
0,1,0.000000,0.000000
1,2,0.408172,0.094340
2,3,0.710859,0.133648
3,4,0.787031,0.147799
4,5,0.799689,0.154088


In [ ]:
runs_wckts_plot = [
        go.Bar(
            name="Runs_per_over",
            x=runs_and_wickets["over"],
            y=runs_and_wickets["total_runs"],
            offsetgroup=0,
        ),
        go.Bar(
            name="Wickets_per_over",
            x=runs_and_wickets["over"],
            y=runs_and_wickets["is_wicket"],
            offsetgroup=1,
        )
        ]
layout = go.Layout(   autosize=False,
                      width=1000,
                      height=600,
                      title='Runs and Wickets per over',
                      xaxis = dict(title='Overs'),
                      yaxis = dict(title='Normalizaed Values'),
                      margin=go.layout.Margin(
                                                    l=50,
                                                    r=50,
                                                    b=100,
                                                    t=100,
                                                    pad = 4
                                                )
                       )
fig = go.Figure(data=runs_wckts_plot, layout=layout)

pyo.iplot(fig)

# Runs Per Match

In [ ]:
Runs_per_match = df_balls.groupby(by='id').sum()['total_runs'].reset_index()
Runs_per_match.head()

,id,total_runs
0,335982,304
1,335983,447
2,335984,261
3,335985,331
4,335986,222


In [ ]:
Runs_per_match_plot = go.Histogram(
                    x=Runs_per_match['total_runs'],                   
                    name='Runs'
                    )
layout = go.Layout(   autosize=False,
                      width=1000,
                      height=600,
                      title='Runs Per Match',
                      xaxis = dict(title='Runs'),
                      yaxis = dict(title='Frequency'),template = 'presentation',
                      # margin=go.layout.Margin(
                      #                               l=50,
                      #                               r=50,
                      #                               b=100,
                      #                               t=100,
                      #                               pad = 4
                      #                           )
                       )
fig = go.Figure(data=[Runs_per_match_plot], layout=layout)

pyo.iplot(fig)

# Most wins

In [28]:
df_matches.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bengaluru,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bengaluru,Kolkata Knight Riders,Royal Challengers Bengaluru,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bengaluru,Mumbai Indians,bat,Royal Challengers Bengaluru,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [40]:
most_wins_plot = [go.Bar(
    x = df_matches['winner'].value_counts().index,
    y = df_matches['winner'].value_counts().values,
    marker = dict(color = random_colors(len(df_matches['winner'].value_counts().index)),line=dict(color='#000000', width=1))
)]

layout = go.Layout(
    autosize=False,
                      width=1000,
                      height=600,
                      title='Most Wins by a team',
                      xaxis = dict(title='Teams'),
                      yaxis = dict(title='Wins'),template = 'presentation',
                                         margin=go.layout.Margin(
                                                    l=100,
                                                    r=50,
                                                    b=200,
                                                    t=100,
                                                    pad = 4
                                                )
)

fig = go.Figure(data=most_wins_plot,layout = layout)
iplot(fig)

# Most Inflential Player

In [41]:
most_mom_plot = [go.Bar(
    x = df_matches['player_of_match'].value_counts()[:20].index,
    y = df_matches['player_of_match'].value_counts()[:20].values,
    marker = dict(color = random_colors(20),line=dict(color='#000000', width=1.5))
)]


layout = go.Layout(
    autosize=False,
                      width=1000,
                      height=600,
                      title='Most Man of the Matches',
                      xaxis = dict(title='Players'),
                      yaxis = dict(title='MoM Awards'),template = 'presentation',
                                         margin=go.layout.Margin(
                                                    l=100,
                                                    r=50,
                                                    b=200,
                                                    t=100,
                                                    pad = 4
                                                )
)

fig = go.Figure(data=most_mom_plot, layout=layout)
iplot(fig)

# Cities as Top Host

In [50]:
matches_citites= df_matches.groupby('city').apply(lambda x:x['city'].count()).reset_index(name='Total Matches')
top_order=matches_citites.sort_values(by='Total Matches',ascending=False)
top=top_order[:10]

most_cities = [go.Pie(labels = top_cities.city.values, values =top_cities["Match Counts"].values,
               marker = dict(colors = random_colors(10),line=dict(color='#000000', width=1.5))) ]

layout = go.Layout(
    autosize=False,
                      width=1000,
                      height=600,
                      title='Most Cities to host matches',
            template = 'presentation',
                                         margin=go.layout.Margin(
                                                    l=100,
                                                    r=50,
                                                    b=200,
                                                    t=100,
                                                    pad = 4
                                                )
)

fig = go.Figure(data=most_cities,layout = layout)
iplot(fig)